## Subtractive alignment + CQT

This notebook uses the alignment between the CQT representations of the part and the full mix recordings to perform the subtractive alignment algorithm.

In [1]:
%matplotlib inline
%load_ext Cython

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import pickle
from numba import njit, prange
import librosa as lb
import librosa.display
from skimage.filters import threshold_triangle
import time
import scipy
import seaborn as sns
import os
import multiprocessing
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn import mixture

In [3]:
def calculate_cqt(audio, sr = 22050, hop_length = 512, bins = 12):
    return np.abs(lb.core.cqt(audio, n_bins = 8 * bins, bins_per_octave = bins, norm = 2))

In [4]:
def get_audio(path):
    audio, sr = lb.core.load(path)
    return audio

Load in all the audio recordings.

In [57]:
piano1_audio = get_audio("data/Audio/piano/piano3.m4a")
violin1_audio = get_audio('data/Audio/violin/violin3.mp3')
cello1_audio = get_audio('data/Audio/cello/cello3.mp3')
# piano2_audio = get_audio("data/Audio/piano/piano4.m4a")
# violin2_audio = get_audio('data/Audio/violin/violin4.mp3')
# cello2_audio = get_audio('data/Audio/cello/cello4.mp3')
fullmix_audio_1 = get_audio('data/Audio/fullmix/fullmix3.mp3')
fullmix_audio_2 = get_audio('data/Audio/fullmix/fullmix4.mp3')

/home/kji/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/kji/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/kji/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/kji/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/kji/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


### Subsequence DTW

In [6]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

import sys
import time


DTYPE_INT32 = np.int32
ctypedef np.int32_t DTYPE_INT32_t

DTYPE_FLOAT = np.float64
ctypedef np.float64_t DTYPE_FLOAT_t

cdef DTYPE_FLOAT_t MAX_FLOAT = float('inf')

# careful, without bounds checking can mess up memory - also can't use negative indices I think (like x[-1])
@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_Cost_To_AccumCostAndSteps(Cin, parameter):
    '''
    Inputs
        C: The cost Matrix
    '''


    '''
    Section for checking and catching errors in the inputs
    '''

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] C
    try:
        C = np.array(Cin, dtype=DTYPE_FLOAT)
    except TypeError:
        print(bcolors.FAIL + "FAILURE: The type of the cost matrix is wrong - please pass in a 2-d numpy array" + bcolors.ENDC)
        return [-1, -1, -1]
    except ValueError:
        print(bcolors.FAIL + "FAILURE: The type of the elements in the cost matrix is wrong - please have each element be a float (perhaps you passed in a matrix of ints?)" + bcolors.ENDC)
        return [-1, -1, -1]

    cdef np.ndarray[np.uint32_t, ndim=1] dn
    cdef np.ndarray[np.uint32_t, ndim=1] dm
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] dw
    # make sure dn, dm, and dw are setup
    # dn loading and exception handling
    if ('dn'  in parameter.keys()):
        try:

            dn = np.array(parameter['dn'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dn (row steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "The type of the elements in dn (row steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dn = np.array([1, 1, 0], dtype=np.uint32)
    # dm loading and exception handling
    if 'dm'  in parameter.keys():
        try:
            dm = np.array(parameter['dm'], dtype=np.uint32)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dm (col steps) is wrong - please pass in a 1-d numpy array that holds uint32s" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE: The type of the elements in dm (col steps) is wrong - please have each element be a uint32 (perhaps you passed a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.uint32)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        print(bcolors.FAIL + "dm (col steps) was not passed in (gave default value [1,0,1]) " + bcolors.ENDC)
        dm = np.array([1, 0, 1], dtype=np.uint32)
    # dw loading and exception handling
    if 'dw'  in parameter.keys():
        try:
            dw = np.array(parameter['dw'], dtype=DTYPE_FLOAT)
        except TypeError:
            print(bcolors.FAIL + "FAILURE: The type of dw (step weights) is wrong - please pass in a 1-d numpy array that holds floats" + bcolors.ENDC)
            return [-1, -1, -1]
        except ValueError:
            print(bcolors.FAIL + "FAILURE:The type of the elements in dw (step weights) is wrong - please have each element be a float (perhaps you passed ints or a long?). You can specify this when making a numpy array like: np.array([1,2,3],dtype=np.float64)" + bcolors.ENDC)
            return [-1, -1, -1]
    else:
        dw = np.array([1, 1, 1], dtype=DTYPE_FLOAT)
        print(bcolors.FAIL + "dw (step weights) was not passed in (gave default value [1,1,1]) " + bcolors.ENDC)

    
    '''
    Section where types are given to the variables we're going to use 
    '''
    # create matrices to store our results (D and E)
    cdef DTYPE_INT32_t numRows = C.shape[0] # only works with np arrays, use np.shape(x) will work on lists? want to force to use np though?
    cdef DTYPE_INT32_t numCols = C.shape[1]
    cdef DTYPE_INT32_t numDifSteps = np.size(dw)

    cdef unsigned int maxRowStep = max(dn)
    cdef unsigned int maxColStep = max(dm)

    cdef np.ndarray[np.uint32_t, ndim=2] steps = np.zeros((numRows,numCols), dtype=np.uint32)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost = np.ones((maxRowStep + numRows, maxColStep + numCols), dtype=DTYPE_FLOAT) * MAX_FLOAT

    cdef DTYPE_FLOAT_t bestCost
    cdef DTYPE_INT32_t bestCostIndex
    cdef DTYPE_FLOAT_t costForStep
    cdef unsigned int row, col
    cdef unsigned int stepIndex

    '''
    The start of the actual algorithm, now that all our variables are set up
    '''
    # initializing the cost matrix - depends on whether its subsequence DTW
    # essentially allow us to hop on the bottom anywhere (so could start partway through one of the signals)
    if parameter['SubSequence']:
        for col in range(numCols):
            accumCost[maxRowStep, col + maxColStep] = C[0, col]
    else:
        accumCost[maxRowStep, maxColStep] = C[0,0]

    # filling the accumulated cost matrix
    for row in range(maxRowStep, numRows + maxRowStep, 1):
        for col in range(maxColStep, numCols + maxColStep, 1):
            bestCost = accumCost[<unsigned int>row, <unsigned int>col] # initialize with what's there - so if is an entry point, then can start low
            bestCostIndex = 0
            # go through each step, find the best one
            for stepIndex in range(numDifSteps):
                #costForStep = accumCost[<unsigned int>(row - dn[<unsigned int>(stepIndex)]), <unsigned int>(col - dm[<unsigned int>(stepIndex)])] + dw[<unsigned int>(stepIndex)] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                costForStep = accumCost[<unsigned int>((row - dn[(stepIndex)])), <unsigned int>((col - dm[(stepIndex)]))] + dw[stepIndex] * C[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)]
                if costForStep < bestCost:
                    bestCost = costForStep
                    bestCostIndex = stepIndex
            # save the best cost and best cost index
            accumCost[row, col] = bestCost
            steps[<unsigned int>(row - maxRowStep), <unsigned int>(col - maxColStep)] = bestCostIndex

    # return the accumulated cost along with the matrix of steps taken to achieve that cost
    return [accumCost[maxRowStep:, maxColStep:], steps]

@cython.boundscheck(False) # turn off bounds-checking for entire function
def DTW_GetPath(np.ndarray[DTYPE_FLOAT_t, ndim=2] accumCost, np.ndarray[np.uint32_t, ndim=2] stepsForCost, parameter):
    '''

    Parameter should have: 'dn', 'dm', 'dw', 'SubSequence'
    '''

    cdef np.ndarray[unsigned int, ndim=1] dn
    cdef np.ndarray[unsigned int, ndim=1] dm
    cdef np.uint8_t subseq
    # make sure dn, dm, and dw are setup
    if ('dn'  in parameter.keys()):
        dn = parameter['dn']
    else:
        dn = np.array([1, 1, 0], dtype=DTYPE_INT32)
    if 'dm'  in parameter.keys():
        dm = parameter['dm']
    else:
        dm = np.array([1, 0, 1], dtype=DTYPE_INT32)
    if 'SubSequence' in parameter.keys():
        subseq = parameter['SubSequence']
    else:
        subseq = 0

    cdef np.uint32_t numRows
    cdef np.uint32_t numCols
    cdef np.uint32_t curRow
    cdef np.uint32_t curCol
    cdef np.uint32_t endCol
    cdef DTYPE_FLOAT_t endCost

    numRows = accumCost.shape[0]
    numCols = accumCost.shape[1]

    # either start at the far corner (non sub-sequence)
    # or start at the lowest cost entry in the last row (sub-sequence)
    # where all of the signal along the row has been used, but only a 
    # sub-sequence of the signal along the columns has to be used
    curRow = numRows - 1
    if subseq:
        curCol = np.argmin(accumCost[numRows - 1, :])
    else:
        curCol = numCols - 1

    endCol = curCol
    endCost = accumCost[curRow, curCol]

    cdef np.uint32_t curRowStep
    cdef np.uint32_t curColStep
    cdef np.uint32_t curStepIndex


    cdef np.ndarray[np.uint32_t, ndim=2] path = np.zeros((2, numRows + numCols), dtype=np.uint32) # make as large as could need, then chop at the end
    path[0, 0] = curRow
    path[1, 0] = curCol

    cdef np.uint32_t stepsInPath = 1 # starts at one, we add in one before looping
    cdef np.uint32_t stepIndex = 0
    cdef np.int8_t done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)
    while not done:
        if accumCost[curRow, curCol] == MAX_FLOAT:
            print('A path is not possible')
            break

        # you're done if you've made it to the bottom left (non sub-sequence)
        # or just the bottom (sub-sequence)
        # find the step size
        curStepIndex = stepsForCost[curRow, curCol]
        curRowStep = dn[curStepIndex]
        curColStep = dm[curStepIndex]
        # backtrack by 1 step
        curRow = curRow - curRowStep
        curCol = curCol - curColStep
        # add your new location onto the path
        path[0, stepsInPath] = curRow
        path[1, stepsInPath] = curCol
        stepsInPath = stepsInPath + 1
        # check to see if you're done
        done = (subseq and curRow == 0) or (curRow == 0 and curCol == 0)

    # reverse the path (a matrix with two rows) and return it
    return [np.fliplr(path[:, 0:stepsInPath]), endCol, endCost]

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [7]:
def frame_to_time(frame, hop_length = 512, sr = 22050):
    return frame * hop_length / sr

In [8]:
def time_to_frame(time, hop_length = 512, sr = 22050):
    return time * sr / hop_length

In [85]:
def calculateErrors(data, annotfile, gtfile, hop_length = 512, sr = 22050, debug = False):
    """Given an alignment and annotation files for the query and the reference, calculate the error between
       each prediction from the alignment and the groud truth annotation in seconds."""
    wp = np.array(sorted(data, key = lambda x: x[0]))
    query_preds = wp[:, 0]
    ref_preds = wp[:, 1]
    query_to_ref = np.interp(list(range(max(query_preds[-1], ref_preds[-1]) + 1)), query_preds, ref_preds)

    data_gt = np.genfromtxt(gtfile, delimiter=',')
    gt_mapping = {}
    for time, idx in data_gt:
        gt_mapping[idx] = time
        
    data_annot = np.genfromtxt(annotfile, delimiter=',')
    errors = []
    clicks = []
    idxs = []
    for time, idx in data_annot:
        frame = int(np.round(time_to_frame(time)))
        ref_frame = query_to_ref[frame]
        pred_ref_time = frame_to_time(ref_frame)
        if debug:
            clicks.append(pred_ref_time)
        if idx in gt_mapping:
            error = np.abs(pred_ref_time - gt_mapping[idx])
            idxs.append(idx)
            errors.append(error)
    if debug:
        return errors, clicks
    return errors

In [86]:
def get_tolerances(errors, tols):
    """Given a list of errors and a range of tolerances, compute the error rate 
       for each additional increment in tolerance."""
    errors = np.array(errors)
    errorRates = []
    for tol in tols:
        toAdd = np.sum(errors > tol) * 1.0 / len(errors)
        errorRates.append(toAdd)
    return errorRates

### Align part to fullmix

In [93]:
@njit(parallel = True)
def calculate_cost_fast(query, ref):
    """Calculates the negative normalized min cost between the query and reference."""
    m, n1 = query.shape
    m, n2 = ref.shape
    result = np.zeros((n1, n2))
    col_sums = np.zeros(n1)
    for j1 in prange(n1):
        for j2 in prange(n2):
            for i in prange(m):
                col_sums[j1] += query[i, j1]
                result[j1, j2] += min(query[i, j1], ref[i, j2])
                
    for j1 in prange(n1):
        for j2 in prange(n2):
            result[j1, j2] *= -1
            result[j1, j2] /= col_sums[j1]
    return result

In [87]:
def align_part_to_fullmix(query, ref, steps = [1, 1, 1, 2, 2, 1], weights = [1, 1, 2]):
    """Uses subsequence DTW and the negative normalized cost metric to compute an alignment between
       the part and full mix."""
    assert len(steps) % 2 == 0, "The length of steps must be even."
    dn = np.array(steps[::2], dtype=np.uint32)
    dm = np.array(steps[1::2], dtype=np.uint32)
    dw = weights
    subsequence = True
    parameter = {'dn': dn, 'dm': dm, 'dw': dw, 'SubSequence': subsequence}

    # Compute cost matrix
    cost = calculate_cost_fast(query, ref)
    
    # DTW
    [D, s] = DTW_Cost_To_AccumCostAndSteps(cost, parameter)
    [wp, endCol, endCost] = DTW_GetPath(D, s, parameter)

    # Reformat the output
    wp = wp.T[::-1]
    return wp

In [13]:
def time_stretch_part(query, ref, alignment):
    """Uses the alignment computed from DTW to time stretch the 
       query to have the same dimensions as the reference."""
    m, n = ref.shape
    feature_stretch = np.zeros((m, n))
    used = set(alignment[:, 1])
    for query_idx, ref_idx in alignment:
        feature_stretch[:, ref_idx] = query[:, query_idx]
    for j in range(n):
        if j not in used:
            feature_stretch[:, j] = feature_stretch[:, j-1]
    return feature_stretch

In [15]:
@njit(parallel = True)
def subtract_part(stretched_cqt, fullmix_cqt):
    """Subtracts the part CQT from the fullmix CQT elementwise."""
    m, n = stretched_cqt.shape
    
    for i in prange(m):
        for j in prange(n):
            fullmix_cqt[i, j] -= stretched_cqt[i, j]
            fullmix_cqt[i, j] = max(fullmix_cqt[i, j], 0)
    return fullmix_cqt

In [16]:
def stretch_segments(segments, wp):
    """Uses the alignment created from DTW to also time stretch the nonsilence segments accordingly."""
    wp = np.array(sorted(wp, key = lambda x: x[0]))
    query_preds = wp[:, 0]
    ref_preds = wp[:, 1]
    query_to_ref = np.interp(list(range(max(query_preds[-1], ref_preds[-1]) + 1)), query_preds, ref_preds)
    n = len(query_to_ref) - 1
    segments[-1][1] = min(segments[-1][1], n)
    return [[int(query_to_ref[a]), int(query_to_ref[b])] for (a, b) in segments]

In [88]:
def weight_segments(segments, part_cqt, fullmix_cqt):
    """Given a set of nonsilence segments, a part CQT, and a full mix CQT, uses an iterative algorithm
       to find the optimal reweighting factor for each segment and scales the segment in the part CQT
       by that factor."""
    alphas = np.concatenate([np.linspace(0.1, 1.0, num = 20), np.arange(1, 11, 0.3), np.arange(10, 510, 10)])
    for segment in segments:
        part_segment = part_cqt[:, segment[0]: segment[1] + 1]
        fullmix_segment = fullmix_cqt[:, segment[0]: segment[1] + 1]
        assert part_segment.shape == fullmix_segment.shape
        best = float('-inf')
        result = 0
        for alpha in alphas:
            val = np.sum(np.minimum(part_segment*alpha, fullmix_segment) - np.maximum(part_segment*alpha - fullmix_segment, 0))
            if val > best:
                best = val
                result = alpha
        part_cqt[:, segment[0]: segment[1] + 1] *= result

In [89]:
def ssa(part_cqt, fullmix_cqt, segments = []):
    """Performs the subtractive alignment algorithm between the part CQT and the full mix CQT
       by first time stretching the part CQT, then performing reweighting, and then subtracting
       the part CQT from the full mix CQT."""
    wp = align_part_to_fullmix(part_cqt, fullmix_cqt)
    stretched_part = time_stretch_part(part_cqt, fullmix_cqt, wp)
    if segments:
        stretched_segments = stretch_segments(segments, wp)
        weight_segments(stretched_segments, stretched_part, fullmix_cqt)
    subtract_part(stretched_part, fullmix_cqt)
    return fullmix_cqt, wp

### Get nonsilence segments in the parts

In [90]:
def get_silence_intervals(silence_indices):
    """Uses a hard silence detection approach to identify contiguous regions
       of nonsilence."""
    cur_interval = []
    start = silence_indices[0]
    for i in range(len(silence_indices) - 1):
        if silence_indices[i] + 1 != silence_indices[i+1]:
            cur_interval.append((start, silence_indices[i]))
            start = silence_indices[i+1]
    cur_interval.append((start, silence_indices[-1]))
    silence_intervals = []
    for start, end in cur_interval:
        start_time = frame_to_time(start)
        end_time = frame_to_time(end)
        if end_time - start_time < 2:
            continue
        silence_intervals.append([start, end])
    return silence_intervals

In [91]:
def get_threshold(total_energies):
    """Uses a Gaussian mixture model fitted to an array of total energies
       to generate a threshold for nonsilence."""
    model = mixture.GaussianMixture(n_components=3, covariance_type="full")
    model.fit(total_energies)
    pi, mu, sigma = model.weights_.flatten(), model.means_.flatten(), np.sqrt(model.covariances_.flatten())
    max_idx = np.argmax(mu)
    threshold = mu[max_idx] - 4 * sigma[max_idx]
    return threshold

In [92]:
def get_segments(audio, H=512, N=2048):
    """Given a piece of audio, calculates all the nonsilence segments
       within the audio using a hard silence detection approach with GMMs."""
    stft = librosa.stft(audio, n_fft=N, hop_length=H)
    energies = np.sum(np.square(abs(stft)), axis=0)
    L = 32
    total_energies = []
    for i in range(len(energies)-L):
        total_energies.append(sum(energies[i:i+L]))
        
    total_energies = np.log(total_energies).reshape(-1, 1)
    threshold = get_threshold(total_energies)
    
    is_silence = [False] * (L//2 - 1)
    for energy in total_energies:
        if energy <= threshold:
            is_silence.append(True)
        else:
            is_silence.append(False)
    is_silence.extend([False] * (L//2))
    silence_indices = np.where(np.array(is_silence) == True)[0]
    silence_intervals = get_silence_intervals(silence_indices)
    nonsilence_segments = []
    cur = 0
    for start, end in silence_intervals:
        nonsilence_segments.append([cur, start])
        cur = end + 1
    nonsilence_segments.append([cur, len(is_silence)])
    return nonsilence_segments

The piano has very few silence regions so we just split it into 5 second segments.

In [22]:
def get_segments_piano(piano_cqt):
    n = piano_cqt.shape[1]
    return [[i, min(i+215, n)] for i in range(0, n, 215)]

In [23]:
# 215 frames = 5 seconds

In [58]:
piano1_cqt = calculate_cqt(piano1_audio)
violin1_cqt = calculate_cqt(violin1_audio)
cello1_cqt = calculate_cqt(cello1_audio)
piano2_cqt = calculate_cqt(piano2_audio)
violin2_cqt = calculate_cqt(violin2_audio)
cello2_cqt = calculate_cqt(cello2_audio)
fullmix1_cqt = calculate_cqt(fullmix_audio_1)
fullmix2_cqt = calculate_cqt(fullmix_audio_2)

In [59]:
nonsilence_piano1 = get_segments_piano(piano1_cqt)
nonsilence_violin1 = get_segments(violin1_audio)
nonsilence_cello1 = get_segments(cello1_audio)
nonsilence_piano2 = get_segments_piano(piano2_cqt)
nonsilence_violin2 = get_segments(violin2_audio)
nonsilence_cello2 = get_segments(cello2_audio)

### Perform subtractive alignment for all 16 combinations in the train/test set

In [60]:
import itertools

In [61]:
arrays = [(0, 1), (0, 1), (0, 1), (0, 1)]

In [62]:
indices = list(itertools.product(*arrays))

In [63]:
piano_segments = [nonsilence_piano1, nonsilence_piano2]
violin_segments = [nonsilence_violin1, nonsilence_violin2]
cello_segments = [nonsilence_cello1, nonsilence_cello2]

In [64]:
piano_cqts = [piano1_cqt, piano2_cqt]
violin_cqts = [violin1_cqt, violin2_cqt]
cello_cqts = [cello1_cqt, cello2_cqt]
fullmix_audio = [fullmix_audio_1, fullmix_audio_2]

If using the training set, we set increment = 1, for the test set, we set increment = 3.

In [65]:
increment = 3

In [34]:
for i, j, k, l in indices:
    piano_cqt, cello_cqt, violin_cqt, fullmix_cqt = piano_cqts[i], cello_cqts[j], violin_cqts[k], calculate_cqt(fullmix_audio[l])
    piano_segs, cello_segs, violin_segs = piano_segments[i], cello_segments[j], violin_segments[k]
    
    fullmix_cqt, piano_fullmix = ssa(piano_cqt, fullmix_cqt, piano_segs)
    fullmix_cqt, violin_fullmix = ssa(violin_cqt, fullmix_cqt, violin_segs)
    fullmix_cqt, cello_fullmix = ssa(cello_cqt, fullmix_cqt, cello_segs)
    
    piano_errs = calculateErrors(piano_fullmix, 
                                 f'data/Annotations/merged/merged/piano{i+increment}.csv', 
                                 f'data/Annotations/merged/merged/fullmix{l+increment}.csv')
    cello_errs = calculateErrors(cello_fullmix,
                                  f'data/Annotations/merged/merged/cello{j+increment}.csv', 
                                  f'data/Annotations/merged/merged/fullmix{l+increment}.csv')
    violin_errs = calculateErrors(violin_fullmix,
                                 f'data/Annotations/merged/merged/violin{k+increment}.csv', 
                                 f'data/Annotations/merged/merged/fullmix{l+increment}.csv')
    
    piano_tols = get_tolerances(piano_errs, np.arange(0,1,1/1000))
    violin_tols = get_tolerances(violin_errs, np.arange(0,1,1/1000))
    cello_tols = get_tolerances(cello_errs, np.arange(0,1,1/1000))
    
    with open(f"tolerances/violin_test_alt/p{i+increment}_c{j+increment}_v{k+increment}_1a_reweight_f{l+increment}.pkl", "wb") as f:
        pickle.dump(violin_tols, f)
    with open(f"tolerances/piano_test_alt/p{i+increment}_c{j+increment}_v{k+increment}_1a_reweight_f{l+increment}.pkl", "wb") as f:
        pickle.dump(piano_tols, f)
    with open(f"tolerances/cello_test_alt/p{i+increment}_c{j+increment}_v{k+increment}_1a_reweight_f{l+increment}.pkl", "wb") as f:
        pickle.dump(cello_tols, f)
        
    print(f"finished {i, j, k, l}")

finished (0, 0, 0, 0)
finished (0, 0, 0, 1)
finished (0, 0, 1, 0)
finished (0, 0, 1, 1)
finished (0, 1, 0, 0)
finished (0, 1, 0, 1)
finished (0, 1, 1, 0)
finished (0, 1, 1, 1)
finished (1, 0, 0, 0)
finished (1, 0, 0, 1)
finished (1, 0, 1, 0)
finished (1, 0, 1, 1)
finished (1, 1, 0, 0)
finished (1, 1, 0, 1)
finished (1, 1, 1, 0)
finished (1, 1, 1, 1)
